In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import pandas as pd
# %% Packages and R functions
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import Formula


grdevices = importr('grDevices')
partykit = importr('partykit')

# Display all columns in the notebook
pd.set_option('display.max_columns', None)

In [2]:
!ls ../data/

CustomerData_LeadGenerator.csv
sample_temperature_data_for_coding_challenge.csv


In [3]:
!python -m rpy2.situation

rpy2 version:
3.5.16
Python version:
3.9.8 (main, Aug 23 2023, 14:14:02) 
[GCC 9.4.0]
Looking for R's HOME:
    Environment variable R_HOME: /usr/lib/R
    Calling `R RHOME`: /usr/lib/R
    Environment variable R_LIBS_USER: ~/R/x86_64-pc-linux-gnu-library/3.6
R's value for LD_LIBRARY_PATH:
/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server:/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server:
R version:
    In the PATH: R version 3.6.3 (2020-02-29) -- "Holding the Windsock"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/usr/share/R/include']
  libraries:
  ['R', 'pcre2-8', 'pcre', 'lzma', 'bz2', 'z', 'rt', 'dl', 'm', 'icuuc', 'icui18n']
  library_dirs:
  ['/usr/lib/R/lib']
  extra_compile_args:
  ['-std=c99']
  extra_link_args:
  ['-Wl,--export-dynamic', '-fopenmp', '-Wl,-Bsymbolic-functions', '-Wl,-z,relro']
Directory for the R shared library:
lib
CFFI

In [2]:
df1 = pd.read_csv('../data/CustomerData_LeadGenerator.csv')
df2 = pd.read_csv('../data/sample_temperature_data_for_coding_challenge.csv')

In [3]:
df1.head()

,fakeID,b_specialisation_i,b_specialisation_h,b_specialisation_g,b_specialisation_f,b_specialisation_e,b_specialisation_d,b_specialisation_c,b_specialisation_b,b_specialisation_a,b_specialisation_j,q_OpeningDays,q_OpeningHours,q_2017 Average Household Size,q_2017 Total Households,q_2017 HHs: 5th Quintile (68.759 and above),q_5th Quint by Total HH,q_2017 Purchasing Power: Per Capita,q_2017 Total Population,"q_2017 Pop 15+/Edu: University, Fachhochschule",q_Uni by Total Pop,q_2017 Personal Care: Per Capita,q_2017 Medical Products: Per Capita,q_2017 Personal Effects: Per Capita,b_in_kontakt_gewesen,b_gekauft_gesamt
0,1069_7334,0,0,0,0,0,0,0,0,0,1,4,36,2.1,1045481,195228,0.186735,23730.35,2194057,324656,0.147971,593.19,326.28,235.75,1,1
1,328_2350,0,0,0,1,1,1,0,0,0,0,4,20,2.1,935739,168291,0.179848,23343.26,1966798,281130,0.142938,588.54,320.36,230.66,1,1
2,865_6285,0,0,0,1,0,1,0,0,0,1,0,0,2.1,1026235,189787,0.184935,23674.18,2149624,320326,0.149015,592.88,325.57,234.82,1,1
3,47_307,0,1,1,0,0,0,0,0,0,0,4,20,2.1,1027701,189634,0.184523,23572.26,2161445,310981,0.143876,590.84,324.06,233.81,1,0
4,342_2513,0,0,0,0,0,1,0,0,0,1,5,33,2.1,1093324,207590,0.189871,23821.63,2302959,334940,0.145439,593.56,327.75,237.41,1,1


In [4]:
df2.head()

,source_id,datetime,property_name,temperature
0,MICDEV001,2019-04-13T17:51:16.000+0000,heating_temperature,33.3
1,MICDEV001,2019-04-13T17:51:16.000+0000,cooling_temperature,15.0
2,MICDEV001,2019-04-13T18:51:18.000+0000,heating_temperature,34.0
3,MICDEV001,2019-04-13T19:51:20.000+0000,heating_temperature,33.8
4,MICDEV001,2019-04-13T20:51:21.000+0000,heating_temperature,34.2


In [5]:
df2.property_name.value_counts()

property_name
heating_temperature    699
cooling_temperature    301
Name: count, dtype: int64

In [6]:
df2.temperature.isna().sum()

0

In [7]:
df1.b_in_kontakt_gewesen.value_counts()

b_in_kontakt_gewesen
0    3678
1      95
Name: count, dtype: int64

In [8]:
df1.b_gekauft_gesamt.value_counts()

b_gekauft_gesamt
0    3716
1      57
Name: count, dtype: int64

In [9]:
df1.columns

Index(['fakeID', 'b_specialisation_i', 'b_specialisation_h',
       'b_specialisation_g', 'b_specialisation_f', 'b_specialisation_e',
       'b_specialisation_d', 'b_specialisation_c', 'b_specialisation_b',
       'b_specialisation_a', 'b_specialisation_j', 'q_OpeningDays',
       'q_OpeningHours', 'q_2017 Average Household Size',
       'q_2017 Total Households',
       'q_2017 HHs: 5th Quintile (68.759 and above)',
       'q_5th Quint by Total HH', 'q_2017 Purchasing Power: Per Capita',
       'q_2017 Total Population',
       'q_2017 Pop 15+/Edu: University, Fachhochschule', 'q_Uni by Total Pop',
       'q_2017 Personal Care: Per Capita',
       'q_2017 Medical Products: Per Capita',
       'q_2017 Personal Effects: Per Capita', 'b_in_kontakt_gewesen',
       'b_gekauft_gesamt'],
      dtype='object')

In [10]:
df1.groupby('b_in_kontakt_gewesen')['b_gekauft_gesamt'].value_counts()


b_in_kontakt_gewesen  b_gekauft_gesamt
0                     0                   3678
1                     1                     57
                      0                     38
Name: count, dtype: int64

In [11]:
df1.groupby(['b_in_kontakt_gewesen', 'b_gekauft_gesamt']).size().reset_index(name='counts')

,b_in_kontakt_gewesen,b_gekauft_gesamt,counts
0,0,0,3678
1,1,0,38
2,1,1,57


In [12]:
df1.describe()

,b_specialisation_i,b_specialisation_h,b_specialisation_g,b_specialisation_f,b_specialisation_e,b_specialisation_d,b_specialisation_c,b_specialisation_b,b_specialisation_a,b_specialisation_j,q_OpeningDays,q_2017 Average Household Size,q_2017 Total Households,q_2017 HHs: 5th Quintile (68.759 and above),q_5th Quint by Total HH,q_2017 Purchasing Power: Per Capita,q_2017 Total Population,"q_2017 Pop 15+/Edu: University, Fachhochschule",q_Uni by Total Pop,q_2017 Personal Care: Per Capita,q_2017 Medical Products: Per Capita,q_2017 Personal Effects: Per Capita,b_in_kontakt_gewesen,b_gekauft_gesamt
count,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3.773000e+03,3773.000000,3773.000000,3773.000000,3.773000e+03,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000,3773.000000
mean,0.218129,0.224225,0.254704,0.111582,0.285449,0.005566,0.002650,0.020408,0.138616,0.254174,3.230586,2.242221,3.873879e+05,73575.237477,0.197419,23170.651983,8.257315e+05,111252.936920,0.101737,574.247339,326.438110,234.848630,0.025179,0.015107
std,0.413030,0.417126,0.435753,0.314894,0.451688,0.074407,0.051421,0.141411,0.345592,0.435454,2.070304,0.153143,4.116316e+05,75533.849191,0.024973,1147.978703,8.565966e+05,131837.721022,0.039749,22.966321,11.135183,11.018168,0.156689,0.121996
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.718000e+03,316.000000,0.100814,19171.140000,3.807000e+03,172.000000,0.027974,490.390000,270.110000,196.450000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.100000,7.571800e+04,15441.000000,0.184566,22662.050000,1.780610e+05,11431.000000,0.063095,560.130000,322.380000,231.310000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.200000,1.537120e+05,34165.000000,0.195882,23627.240000,3.417850e+05,39282.000000,0.101592,584.000000,325.860000,235.290000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,2.400000,9.651600e+05,174038.000000,0.213667,23839.540000,2.023342e+06,298968.000000,0.145630,592.520000,331.930000,238.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,2.900000,1.113261e+06,212126.000000,0.269397,26008.530000,2.350279e+06,337338.000000,0.170786,610.140000,363.230000,274.140000,1.000000,1.000000


In [13]:
df_training

NameError: name 'df_training' is not defined

In [14]:
# Group by 'target' and calculate mean for binary columns
binary_cols = [ 'b_specialisation_i', 'b_specialisation_h',
       'b_specialisation_g', 'b_specialisation_f', 'b_specialisation_e',
       'b_specialisation_d', 'b_specialisation_c', 'b_specialisation_b',
       'b_specialisation_a', 'b_specialisation_j']
distribution_by_target = df_training.groupby('b_gekauft_gesamt')[binary_cols].mean()

print("Distribution of binary columns based on the target:")
print(distribution_by_target)

NameError: name 'df_training' is not defined

In [291]:
df_training[df_training['b_gekauft_gesamt']==1]['b_specialisation_b'].value_counts()

b_specialisation_b
0    54
1     3
Name: count, dtype: int64

In [292]:
df_training[df_training['b_specialisation_b']==0]['b_gekauft_gesamt'].value_counts()

b_gekauft_gesamt
1    54
0    38
Name: count, dtype: int64

In [ ]:
b_specialisation_b ==0 then  b_gekauft_gesamt==0

In [15]:
df_training = df1[lambda x: x["b_in_kontakt_gewesen"]==1].copy()
# Perform stratified sampling to get 50% of the data
X = df_training.drop(columns=['b_gekauft_gesamt'])  # Features
y = df_training['b_gekauft_gesamt']  # Target variable

# Split the data into training and testing sets, stratifying by y
X_sampled, _, y_sampled, _ = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

# Concatenate X_sampled and y_sampled to get the sampled DataFrame
df_sampled = pd.concat([X_sampled, y_sampled], axis=1)

In [16]:
df_sampled['b_gekauft_gesamt'].value_counts()


b_gekauft_gesamt
1    28
0    19
Name: count, dtype: int64

In [17]:
# R functions
ctree_control = partykit.ctree_control
ctree = partykit.ctree
glmtree = partykit.glmtree
plot = ro.r['plot']

predict = ro.r['predict']

In [18]:
# Target
target = 'b_gekauft_gesamt'

In [19]:
# Filters
add_filters = (
    (df_sampled[target].notna())
)

data = df_sampled.loc[add_filters].copy()
data.columns = data.columns.str.replace(' ', '_')
# Replace dots, colons, and plus symbols in column names
data.columns = data.columns.str.replace(r'[\.\:\+/()\[\]\\,]', '', regex=True)

In [20]:
data.columns.tolist()

['fakeID',
 'b_specialisation_i',
 'b_specialisation_h',
 'b_specialisation_g',
 'b_specialisation_f',
 'b_specialisation_e',
 'b_specialisation_d',
 'b_specialisation_c',
 'b_specialisation_b',
 'b_specialisation_a',
 'b_specialisation_j',
 'q_OpeningDays',
 'q_OpeningHours',
 'q_2017_Average_Household_Size',
 'q_2017_Total_Households',
 'q_2017_HHs_5th_Quintile_68759_and_above',
 'q_5th_Quint_by_Total_HH',
 'q_2017_Purchasing_Power_Per_Capita',
 'q_2017_Total_Population',
 'q_2017_Pop_15Edu_University_Fachhochschule',
 'q_Uni_by_Total_Pop',
 'q_2017_Personal_Care_Per_Capita',
 'q_2017_Medical_Products_Per_Capita',
 'q_2017_Personal_Effects_Per_Capita',
 'b_in_kontakt_gewesen',
 'b_gekauft_gesamt']

In [21]:
# Original list of column names
columns = data.columns.tolist()

# Transform the column names with quotes and +
transformed_columns = ['"+ ' + col + '"' for col in columns]

# Print each transformed column name
for col in transformed_columns:
    print(col)

"+ fakeID"
"+ b_specialisation_i"
"+ b_specialisation_h"
"+ b_specialisation_g"
"+ b_specialisation_f"
"+ b_specialisation_e"
"+ b_specialisation_d"
"+ b_specialisation_c"
"+ b_specialisation_b"
"+ b_specialisation_a"
"+ b_specialisation_j"
"+ q_OpeningDays"
"+ q_OpeningHours"
"+ q_2017_Average_Household_Size"
"+ q_2017_Total_Households"
"+ q_2017_HHs_5th_Quintile_68759_and_above"
"+ q_5th_Quint_by_Total_HH"
"+ q_2017_Purchasing_Power_Per_Capita"
"+ q_2017_Total_Population"
"+ q_2017_Pop_15Edu_University_Fachhochschule"
"+ q_Uni_by_Total_Pop"
"+ q_2017_Personal_Care_Per_Capita"
"+ q_2017_Medical_Products_Per_Capita"
"+ q_2017_Personal_Effects_Per_Capita"
"+ b_in_kontakt_gewesen"
"+ b_gekauft_gesamt"


In [22]:
data["q_OpeningDays"]

104    4
72     5
99     1
75     3
46     2
3      4
79     0
53     4
74     1
41     2
16     4
55     0
101    5
38     4
31     0
52     5
30     3
39     0
18     5
45     0
60     4
92     5
68     5
90     2
0      4
2      0
49     4
95     0
19     3
34     5
80     0
26     0
84     5
77     5
29     4
36     4
71     5
73     0
22     5
11     0
44     3
86     5
51     5
21     4
5      4
78     0
89     0
Name: q_OpeningDays, dtype: int64

In [23]:
from rpy2.rinterface_lib.embedded import RRuntimeError
#Add base prediction to test corrector model
data['base_pred'] = 1 # - data['score_value_replay'] / 10000


# Convert to R dataframe
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)
    r_base_pred = ro.conversion.py2rpy(data['base_pred'])

# %%
p_value = 0.4
controls = ctree_control(mincriterion=1 - p_value, minbucket=10)

# Just to be sure it ran properly
if 'ct' in globals():
    del ct

ct = ctree(
    formula=Formula(
        'factor(' + target + ') ~ '
                             # 'score_value_replay'
                             # '+ mean_smoothness'
                             # '+ fakeID'
                            '+ b_specialisation_i'
                            '+ b_specialisation_h'
                            '+ b_specialisation_g'
                            '+ b_specialisation_f'
                            '+ b_specialisation_e'
                            '+ b_specialisation_d'
                            '+ b_specialisation_c'
                            '+ b_specialisation_b'
                            '+ b_specialisation_a'
                            '+ b_specialisation_j'
                            # '+ q_OpeningDays'
                            # '+ q_OpeningHours'
                            '+ q_2017_Average_Household_Size'
                            '+ q_2017_Total_Households'
                            '+ q_2017_HHs_5th_Quintile_68759_and_above'
                            '+ q_5th_Quint_by_Total_HH'
                            '+ q_2017_Purchasing_Power_Per_Capita'
                            '+ q_2017_Total_Population'
                            '+ q_2017_Pop_15Edu_University_Fachhochschule'
                            '+ q_Uni_by_Total_Pop'
                            '+ q_2017_Personal_Care_Per_Capita'
                            '+ q_2017_Medical_Products_Per_Capita'
                            '+ q_2017_Personal_Effects_Per_Capita'
    ),
    data=r_data,
    control=controls
)
# Define the formula dynamically
# target = 'target'  # Replace with your actual target variable name
# formula_str = f'factor({target}) ~ ' + \
#               '+'.join([
#                   'b_specialisation_i', 'b_specialisation_h',
#                   'b_specialisation_g', 'b_specialisation_f',
#                   'b_specialisation_e', 'b_specialisation_d',
#                   'b_specialisation_c', 'b_specialisation_b',
#                   'b_specialisation_a', 'b_specialisation_j',
#                   'q_2017_Medical_Products_Per_Capita'
#               ])

# # Run ctree model in R
# try:
#     # Ensure the 'party' package is installed in R
#     party = importr('party')

#     # Set controls for the ctree model
#     p_value = 0.5
#     controls = ro.r.ctree_control(mincriterion=1 - p_value, minbucket=100)

#     # Fit ctree model
#     ct = ro.r.ctree(
#         formula=ro.Formula(formula_str),
#         data=r_data,
#         control=controls
#     )

#     # Print the model summary
#     print(ct)

#     # Plot the tree
#     grdevices = importr('grDevices')
#     grdevices.png(file="./results/default_ctree_pt.png", width=2500, height=500)
#     ro.r.plot(ct, type='simple')
#     grdevices.dev_off()

# except RRuntimeError as e:
#     print(e)


# %% Plot tree
grdevices.png(file="./results/default_ctree_pt.png", width=2500, height=500)
plot(ct, type='simple')
grdevices.dev_off()

1


In [24]:
df1[lambda x: x["b_in_kontakt_gewesen"]==0]["b_specialisation_g"].value_counts()

b_specialisation_g
0    2760
1     918
Name: count, dtype: int64

In [25]:
!pwd

/home/lisbeth/Documents/Adelchi/customer_generator_temperature_readings/notebooks
